<a href="https://colab.research.google.com/github/kimchanyoo/Data_Science/blob/main/Pandas4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#피봇테이블

In [5]:
import pandas as pd
import numpy as np
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
# pivot(행 인덱스, 열 인덱스, 인구)
df1.pivot("도시", "연도", "인구")
## pivot 메소드 대신 set_index와 unstack
df1.set_index(["도시","연도"])[["인구"]].unstack()
df1.pivot(["지역","도시"],"연도","인구")
""" 
행 인덱스와 열 인덱스는 데이터를 찾는 Key의 역할의 하는데 Key 값을 충족하는 값이
2개 이상이면 ValueError가 난다
"""

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

# 그룹분석



In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
"""
groupby 메서드는 데이터를 그룹별로 분류하는 역할을 수행
º열 또는 열 리스트
º행 인덱스
"""
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
groups = df2.groupby(df2.key1)
data_sum = df2.data1.groupby(df2.key1).sum()
df3 = pd.DataFrame(data_sum)
df1["인구"].groupby([df1["지역"],df1["연도"]]).sum().unstack("연도")
"""
150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length),
꽃잎폭(sepal_width), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width)을 측정한 데이터
"""
iris = sns.load_dataset("iris")
def peak_to_peak_ratio(x):
  return x.max()/x.min()
# agg 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.
iris.groupby(iris.species).agg(peak_to_peak_ratio)
# apply 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]
iris.groupby(iris.species).apply(top3_petal_length)
# describe 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
iris.groupby(iris.species).describe().T
# translate  그룹별 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터프레임 자체를 변화
def q3cut(s):
  return pd.qcut(s,3,labels=["소","중","대"]).astype(str)
iris["petal_length_class"]=iris.groupby(iris.species).petal_length.transform(q3cut)
iris[["petal_length","petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


## 연습문제 4.7.2

In [21]:
"""
붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 
꽃잎폭(sepal_width) 등의 평균을 구하라. 만약 붓꽃 종(species)이 표시되지 않았을 때 
이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.
"""
iris.groupby(iris.species).mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


#pivot_table

In [43]:
# pivot_table 명령은 그룹분석은 하지만 최종명령은 피봇테이블처럼 한다
"""
pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', 

fill_value=None, margins=False, margins_name='All')

data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)

values: 분석할 데이터프레임에서 분석할 열

index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트

columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트

aggfunc: 분석 메서드

fill_value: NaN 대체 값

margins: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부

margins_name: 마진 열(행)의 이름
"""
# aggfunc 이 주어지지 않으면 기본적으로 aggfunc = mean()을 수행함
df1.pivot_table("인구","도시","연도",margins= True, margins_name="합계")
df1.pivot_table("인구",index=["연도","도시"])

tips = sns.load_dataset("tips")
tips.tail()
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.describe()
tips.groupby("sex").size()
tips.groupby(["sex","smoker"]).size()
tips.pivot_table("tip_pct","sex","smoker",aggfunc="count",margins=True)
tips.groupby("sex")[["tip_pct"]].mean()
tips.pivot_table("tip_pct","sex")
tips.groupby(["sex","smoker"])[["tip_pct"]].describe().T
tips.pivot_table(['tip_pct', 'size'], ['sex', 'day'], 'smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

##연습 문제 4.7.3

In [42]:
"""
팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.

어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?
"""
tips1 = tips.pivot_table('tip_pct',['time','day'],'size',dropna=True)
tips2 = tips.pivot_table('tip_pct',['time','day','size'])
display(tips1,tips2)

size                1         2         3         4         5         6
time   day                                                             
Lunch  Thur  0.181728  0.164024  0.144599  0.145515  0.121389  0.173706
       Fri   0.223776  0.181969  0.187735       NaN       NaN       NaN
Dinner Thur       NaN  0.159744       NaN       NaN       NaN       NaN
       Fri        NaN  0.162659       NaN  0.117750       NaN       NaN
       Sat   0.231832  0.155289  0.151439  0.138289  0.106572       NaN
       Sun        NaN  0.180870  0.152662  0.153168  0.159839  0.103799

tip_pct
time   day  size          
Lunch  Thur 1     0.181728
            2     0.164024
            3     0.144599
            4     0.145515
            5     0.121389
            6     0.173706
       Fri  1     0.223776
            2     0.181969
            3     0.187735
Dinner Thur 2     0.159744
       Fri  2     0.162659
            4     0.117750
       Sat  1     0.231832
            2     0.155289
            3     0.151439
            4     0.138289
            5     0.106572
       Sun  2     0.180870
            3     0.152662
            4     0.153168
            5     0.159839
            6     0.103799

##연습 문제 4.7.4

In [69]:
"""
타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.

titanic = sns.load_dataset("titanic")

qcut 명령으로 세 개의 나이 그룹을 만든다.

성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다. 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.

성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.
"""
titanic = sns.load_dataset("titanic")
titanic["age_cut"] = pd.qcut(titanic.age ,3,labels=["청년","중년","노년"]).astype(str)
def age_ratio(x):
  return x.sum()/len(titanic)
titanic.groupby(["sex","age_cut","class"])[["survived"]].agg(age_ratio).unstack("class")
titanic.groupby(["sex","class"])[["survived"]].agg(age_ratio)

survived
sex    class           
female First   0.968085
       Second  0.921053
       Third   0.500000
male   First   0.368852
       Second  0.157407
       Third   0.135447